In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 7.6 MB/s eta 0:00:00


In [ ]:
!pip install datasets

In [ ]:
from pypdf import PdfReader
import re
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import pipeline



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Function to extract text from PDF
def extract_text(pdf_path):
    combined_text = ""
    reader = PdfReader(pdf_path)
    for page in reader.pages:
        text = page.extract_text()
        if text:
            combined_text += text + "\n"
    return combined_text

pdf_file = r"LASGPSR.pdf"

In [ ]:
# Extract text
dataset_text = extract_text(pdf_file)

# Save extracted text to a file
with open("dataset.txt", "w", encoding="utf-8") as f:
    f.write(dataset_text)

print("Text extraction complete! First 1000 characters:\n")
print(dataset_text[:1000])  # Preview first 1000 characters

Text extraction complete! First 1000 characters:

 
 
 
LAGOS STATE GOVERNMENT 
PUBLIC SERVICE 
RULES  
REVISED TO 1st JANUARY, 2015  

 
 
 
 
 
 Lagos State Government Public Service Rules Foreword 
FOREWORD 
It seems trite to contend that there is nothing that fairly mirrors the image, 
perception and indeed the effectiveness of any Government like the image, perception 
and effectiveness of its Public Service. I have heard people refer to the word 
"bureaucracy" as if it was itself a bad thing. Checking up on the meaning of the word 
confirms what I already knew. It refers to a body of rules, procedures and processes by 
which things are done. 
In my opinion this is a good thing. A large workforce like the Lagos State Public 
Service, inclusive of its Local Government employees, numerous Parastatals and 
Commissions comprising over one hundred thousand people, employed to relate with 
members of the public, provide services and solve problems, require Rules of 
Engagement. Without 

In [ ]:
# Function to clean text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[^\w\s.,!?]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

cleaned_text = clean_text(dataset_text)
print(cleaned_text[:1000])  # Display first 1000 characters after cleaning

LAGOS STATE GOVERNMENT PUBLIC SERVICE RULES REVISED TO 1st JANUARY, 2015 Lagos State Government Public Service Rules Foreword FOREWORD It seems trite to contend that there is nothing that fairly mirrors the image, perception and indeed the effectiveness of any Government like the image, perception and effectiveness of its Public Service. I have heard people refer to the word bureaucracy as if it was itself a bad thing. Checking up on the meaning of the word confirms what I already knew. It refers to a body of rules, procedures and processes by which things are done. In my opinion this is a good thing. A large workforce like the Lagos State Public Service, inclusive of its Local Government employees, numerous Parastatals and Commissions comprising over one hundred thousand people, employed to relate with members of the public, provide services and solve problems, require Rules of Engagement. Without clear processes, rules and procedures, their conduct within and with members of the publ

In [ ]:
print(cleaned_text[:1000])

LAGOS STATE GOVERNMENT PUBLIC SERVICE RULES REVISED TO 1st JANUARY, 2015 Lagos State Government Public Service Rules Foreword FOREWORD It seems trite to contend that there is nothing that fairly mirrors the image, perception and indeed the effectiveness of any Government like the image, perception and effectiveness of its Public Service. I have heard people refer to the word bureaucracy as if it was itself a bad thing. Checking up on the meaning of the word confirms what I already knew. It refers to a body of rules, procedures and processes by which things are done. In my opinion this is a good thing. A large workforce like the Lagos State Public Service, inclusive of its Local Government employees, numerous Parastatals and Commissions comprising over one hundred thousand people, employed to relate with members of the public, provide services and solve problems, require Rules of Engagement. Without clear processes, rules and procedures, their conduct within and with members of the publ

In [ ]:
# Load GPT-2 tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
# Set padding token
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
# Split text into smaller segments before tokenization
chunk_size = 500
text_chunks = [cleaned_text[i : i + chunk_size] for i in range(0, len(cleaned_text), chunk_size)]

# Tokenize each chunk separately
token_chunks = [tokenizer(chunk, truncation=True, padding="max_length", max_length=512)["input_ids"] for chunk in text_chunks]

# Convert to tensor
input_ids = torch.tensor(token_chunks)

# Print tokenized shape
print("Tokenized input shape:", input_ids.shape)


Tokenized input shape: torch.Size([630, 512])


In [ ]:
from torch.utils.data import Dataset, DataLoader

# Define custom dataset class
class TextDataset(Dataset):
    def __init__(self, input_ids):
        self.input_ids = input_ids

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):

        labels = self.input_ids[idx].clone()
        labels[labels == tokenizer.pad_token_id] = -100


        return {"input_ids": self.input_ids[idx], "labels": labels}

train_dataset = TextDataset(input_ids[:train_size])
val_dataset = TextDataset(input_ids[train_size:])

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Print dataset sizes
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

# Test DataLoader
batch = next(iter(train_dataloader))
print("Batch shape:", batch["input_ids"].shape)

Train dataset size: 567
Validation dataset size: 63
Batch shape: torch.Size([8, 512])


In [ ]:
# Load pre-trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

training_args = TrainingArguments(
    output_dir="./gpt2_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=2,  # Reduce batch size
    per_device_eval_batch_size=2,  # Reduce batch size
    num_train_epochs=20,
    logging_dir="./logs",
    save_total_limit=2,  # Keeps only the last 2 checkpoints
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=True  # Enables mixed precision to save memory
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Start training
trainer.train()

# Save final model and tokenizer
model.save_pretrained("./gpt2_finetuned")
tokenizer.save_pretrained("./gpt2_finetuned")


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


Epoch,Training Loss,Validation Loss
1,No log,3.553396
2,3.330600,3.506344
3,3.330600,3.500168
4,2.562400,3.573910
5,2.562400,3.672707
6,2.076400,3.778577
7,2.076400,3.983768
8,1.655800,4.113572
9,1.293700,4.305134
10,1.293700,4.450242


('./gpt2_finetuned/tokenizer_config.json',
 './gpt2_finetuned/special_tokens_map.json',
 './gpt2_finetuned/vocab.json',
 './gpt2_finetuned/merges.txt',
 './gpt2_finetuned/added_tokens.json')

In [ ]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Test the model with a prompt
prompt = "How many days leave does a level 8 officer get?"
output = generator(prompt, max_length=250, num_return_sequences=1)

print("Generated Text:\n", output[0]['generated_text'])


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generated Text:
 How many days leave does a level 8 officer get? b How often does his Leave of Absence be Leave for a calendar year? The calculation is based on working days offduty plus any days in excess of 8 working days which are part of annual leave. 120208 i Average Leave d Money Lending Service is a body which expec ts officers spend their annual leave on various things such as travelling, conference business, domestic servants, accommodation and training. Officers all live in the same house and can borrow from one another. ii An officer must take Care of his Maternity Leave within the calendar year in which it is due. Where it is due within the calendar year, the officer must ilevel himself fro the Maternity Leave, ilevel s not be taken ix months before ix th ult due. 120209 ii Types of Maternity Leave ix Approved ix Spouses Maternity Leave x Spouses Caring for a Paternity Leave xix ive Adoption Leave Lagos State Govt. ix Year XXX xy All Maternity Leave ix YinYang Spare SICK LE

In [ ]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Test the model with a prompt
prompt = "Who is entitled to conversion?"
output = generator(prompt, max_length=500, num_return_sequences=1)

print("Generated Text:\n", output[0]['generated_text'])

Device set to use cuda:0


Generated Text:
 Who is entitled to conversion? 020504 No. 020210 An officer who is promoted shall be treated as proceeding on traditional vernacular courses of instruction that do not overlap his formal educational qualifications. However, his vernacular course of instruction shall be formally part of his formal educational record and he shall be eligible for free transport to and from his new station on normal rates. SECTION 6 LEAVE 020506 Where an officer is required, at the instance of his Permanent SecretaryHead of ExtraMinisterial DepartmentAgency, to leave his station after a tour of 14 months because of shi s inability to attend to his s vernacular vernacular ization, th at officer may ile be allowed lea ve to resume his cadre thereon without ile a break in service ix Any leave pecuniary damage arising from availing himself of ile leave shall be deductible ix Deferred leave shall be limited to the shortest icial period ixii Any ile which ix uri ng a officers s inability to latu

In [ ]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Test the model with a prompt
prompt = "what are the types of journeys?"
output = generator(prompt, max_length=500, num_return_sequences=1)

print("Generated Text:\n", output[0]['generated_text'])

Device set to use cuda:0


Generated Text:
 what are the types of journeys? How does it compare to other journeys undertaken for example, baggage allowance for one spouse and one child on a journey abroad on the instruction of another officer? How does training in one type of vehicle fare in another? What is the transport cost in Lagos State? What is the estimated cost in special rates for other modes of transportation? i.e. fare in gazetted form at the earliest possible date? ii.e. fare in ABSENCEPOINTMENTfo nt format ion, where possible, the following Types of Transport Facilities................................................... a.............................................. 15 Lagos State Government Public Service Rules Chapter 15 CHAPTER 15 DUTY VISITS OUTSIDE NIGERIA Section 1 Domestic Staff Tour...................................................... 150101 Tour..................................................................................................................................................